# Etape 5 : Challenge TETLNET

Telnet utilise le protocole IP, et utilise le port 23.

L'objectif est :

- Retrouver le login/mot de passe d'une connexion telnet dans le fichier de capture proposé
- Ecrire un script Python permettant de récupérer automatiquement les login/ mot de passe dans cette   capture
- Ecrire un script Python permettant de reconnaître une connexion telnet et de récupérer automatiquement les login/mot de passe échangés sur le réseau 
A vrai dire la seule difficulté supplémentaire consiste à repérer comment est transmis le mot de passe, ce qui constitue la seule réelle nouveauté.

#### Test d'affichage partie des lignes concerné :

In [3]:
from scapy.all import *

listetrame=rdpcap("telnet-total.pcapng") #ici on importe notre trame

listetrame.summary()

Ether / ARP who has 10.2.4.4 says 10.2.4.1 / Padding
Ether / ARP who has 10.2.4.4 says 10.2.4.1 / Padding
Ether / IPv6 / UDP / DNS Qry "b'28.122.35.161.in-addr.arpa.'" 
Ether / IP / UDP / DNS Qry "b'28.122.35.161.in-addr.arpa.'" 
Ether / IPv6 / ICMPv6ND_NS / ICMPv6 Neighbor Discovery Option - Source Link-Layer Address 00:13:1a:a3:e1:18
802.3 b0:aa:77:e0:f8:8d > 01:80:c2:00:00:00 / LLC / STP / Padding
Ether / IPv6 / ICMPv6ND_NS / ICMPv6 Neighbor Discovery Option - Source Link-Layer Address 00:13:1a:a3:e1:18
Ether / ARP who has 10.2.4.4 says 10.2.4.1 / Padding
Ether / ARP who has 10.2.4.4 says 10.2.4.1 / Padding
Ether / IPv6 / ICMPv6ND_NS / ICMPv6 Neighbor Discovery Option - Source Link-Layer Address 00:13:1a:a3:e1:18
Ether / IPv6 / UDP 2a00:1450:4006:800::200a:https > 2001:660:6701:30cc:b0c9:8f6f:ed17:90a2:54430 / Raw
Ether / IPv6 / UDP 2a00:1450:4006:800::200a:https > 2001:660:6701:30cc:b0c9:8f6f:ed17:90a2:54430 / Raw
Ether / IPv6 / UDP 2001:660:6701:30cc:b0c9:8f6f:ed17:90a2:54430 > 2a

In [6]:
listetrame[83][Raw] #la trame avec le login trouver dans Wireshark

<Raw  load='734cc26aaf98 login: ' |>

Après avoir visualiser sur wireshard on vois que le login est "marie"

In [7]:
listetrame[118][Raw] #la trame avec le password trouver dans Wireshark

<Raw  load='Password: ' |>

Après avoir visualiser sur wireshark on vois que le mdp est : poppins

In [8]:
listetrame[282][Raw]

<Raw  load='ceci est un fichier indispensable ...\r\n' |>

In [9]:
listetrame[284][Raw]

<Raw  load='\x1b]0;marie@734cc26aaf98: ~\x07\x1bmarie@734cc26aaf98\x1b:\x1b~\x1b$ ' |>

# Programme final : 

In [31]:
from scapy.all import *

listetrame=rdpcap("telnet-total.pcapng") #ici on importe notre trame

presencelogin=0 #pour ensuite verif si le login est la
presencepsw=0 #pour ensuite verif le password est la
login="" #pour stocker tous les caratère du login envoyer 1 par 1 en telnet
psw="" #pour stocker tous les caratère du mots de passe envoyer 1 par 1 en telnet

for elm in listetrame : #pour tout les element dans la trame
    try :
        if elm['Ether'].type==0x0800: #si l'en tête etherne a pour type suivant l'IPv4
            if elm['TCP'].dport == 23 or elm['TCP'].sport == 23: #si le fort source ou destination correspont à 23 donc telnet
                if elm['Raw'].load != '': # Vérifie si les données ne sont pas égale à une chaîne d'octets vide (''). 
                    trametrier=str(elm['Raw'].load) #on associe les donnée a la variable trame trier et on les fait passer en chaine de caractère pour pourvoir faire des "if" ensuite
                    #print(trametrier)

                    if "login:" in trametrier : #ici on regarde si les caractère "login:" sont dans notre trame trié
                        presencelogin=1 #si on trouve login on met la présence login à 1 pour l'utiliser ensuite
                    
                    if "Password:" in trametrier: #ici on regarde si les caractère "Password" sont dans notre trame trié
                        presencepsw=1 #si on trouve "Password" on met la présence pswx à 1 pour l'utiliser ensuite
                    
                    if "\\x00" in trametrier : #si il y a le "flag" dans la trame 
                        presencelogin = 0 #alors on met les présence a 0 pour ne pas continuer le trie que on fera ensuite
                        presencepsw =0
                    
                    if presencelogin==1 and elm['TCP'].dport==23: #si le login est présent et que le port destination de la trame est = à 23 (telnet)
                        login = login + str(trametrier[-2]) #alors on ajouter à la variable login le login avec l'avant dernière partie de la trametrier (donc la lettre)
                    
                    if presencepsw ==1 and elm['TCP'].dport==23: #si le passeword est présent et que le port destination de la trame est = à 23 (telnet)
                        psw = psw + str(trametrier[-2])  #alors on ajouter à la variable psw le password avec l'avant dernière partie de la trametrier (donc la lettre)
                    

    except:
        continue

print("le login est :", login)
print("le mot de passe est :", psw)

le login est : marie
le mot de passe est : poppins
